In [1]:
import os
import cv2
import h5py
import numpy as np
from tqdm.auto import tqdm
from multiprocess import Pool
from toolz import concat
from aging.organization.paths import FOLDERS

In [2]:
n_cpus = int(os.environ.get("SLURM_CPUS_PER_TASK", 1))

In [4]:
target_depth = 670  # mm - ~26.3in

In [5]:
def scale_mouse(scale, input_im):
    '''scale input to match target'''
    if isinstance(scale, np.ndarray):
        scale = scale[0]
    warp_mat = cv2.getRotationMatrix2D((input_im.shape[0] // 2, input_im.shape[1] // 2), 0, scale)
    out_im = cv2.warpAffine(input_im, warp_mat, input_im.shape, flags=cv2.INTER_LINEAR)
    return out_im

In [7]:
depth_files = sorted(concat(f.glob('**/results_00.h5') for f in FOLDERS))

In [12]:
def not_rescaled(file):
    try:
        with h5py.File(file, 'r') as h5f:
            return 'rescaled_frames' not in h5f
    except (BlockingIOError, OSError):
        return False

In [9]:
def rescale(file):
    with h5py.File(file, 'a') as h5f:
        true_depth = h5f['metadata/extraction/true_depth'][()]
        frames = h5f['frames'][()]
        factor = true_depth / target_depth
        # factor = depth_pred(true_depth, target_depth)
        # kornia is 2.5x slower than this
        scaled_frames = np.array([scale_mouse(factor, frame) for frame in frames.astype('float32')])
        h5f.create_dataset('rescaled_frames', data=scaled_frames.astype('uint8'), compression='gzip')
    return True

In [10]:
# for file in tqdm(depth_files):
#     with h5py.File(file, 'a') as h5f:
#         if 'rescaled_frames' in h5f:
#             del h5f['rescaled_frames']

In [13]:
pool = Pool(n_cpus)
for i, file in enumerate(
    tqdm(
        pool.imap_unordered(rescale, filter(not_rescaled, depth_files)),
        total=len(depth_files),
    )
):
    continue

  0%|          | 0/174 [00:00<?, ?it/s]